## Bounding Boxes Convolution Network Training
I love buzzwords.


In [38]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import cv2

In [39]:
%load_ext autoreload
%autoreload 2
import sys
paths = ['../models', '../dataset', '../util']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)

from bounding_boxes import *
from data_generators import get_data_with_bbox_coords
import dataset
from callbacks import ShowSegmentation

bboxes = get_bounding_boxes('../data')
bboxes = largest_bbox_per_image(bboxes)
train_data = dataset.train
val_data = dataset.validation
 
train_gen = get_data_with_bbox_coords(train_data, '../data/train', bboxes,
                                      flip=True, shift_x=5, shift_y=5, rot_range=10,
                                      elastic_trans=True)

val_gen = get_data_with_bbox_coords(val_data, '../data/train', bboxes)
n_labels = len(dataset.labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
def rescale_gen(train_gen):
    while True:
        nxt = next(train_gen)
        yield (nxt[0], (nxt[1].astype(np.float) / nxt[0].shape[3]))

Function to define the layers of the convnet.

In [53]:
def keras_model_conv():
    model = Sequential()
    
    model.add(Convolution2D(16, 3, 3, activation='relu', input_shape=(3, 256, 256)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='relu'))

    model.compile(loss='mse',
              optimizer=Adam(),
              metrics=['accuracy', 'mse'])

    return model

Initialization of the model.

In [54]:
model = keras_model_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_16 (Convolution2D) (None, 16, 254, 254)  448         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
maxpooling2d_16 (MaxPooling2D)   (None, 16, 127, 127)  0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
dropout_19 (Dropout)             (None, 16, 127, 127)  0           maxpooling2d_16[0][0]            
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 64, 125, 125)  9280        dropout_19[0][0]                 
___________________________________________________________________________________________

Fit the model using a generator.

Use: 
fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None, max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0)


In [55]:
model.fit_generator(rescale_gen(train_gen), samples_per_epoch=len(train_data),
                    nb_epoch=50, verbose=1,
                    callbacks=None,
                    validation_data=rescale_gen(val_gen), nb_val_samples=len(val_data))

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/The Mountain/AppData/Local/Theano/compiledir_Windows-8.1-6.3.9600-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.13-64/tmpweb_rc/c249ee98ee90d69e3c3501d08f3438cd.lib and object C:/Users/The Mountain/AppData/Local/Theano/compiledir_Windows-8.1-6.3.9600-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.13-64/tmpweb_rc/c249ee98ee90d69e3c3501d08f3438cd.exp



Epoch 1/50
3040/3021 [==============================] - 155s - loss: 0.0855 - acc: 0.4243 - mean_squared_error: 0.0855 - val_loss: 0.0830 - val_acc: 0.3724 - val_mean_squared_error: 0.0830

KeyboardInterrupt: 

In [8]:
model.save

<bound method Sequential.save of <keras.models.Sequential object at 0x0000000017F675C0>>